# Excercise 1

Demonstrate the importance of distance/similarity metric over the Reuters
Text Categorization Collection. The database is available at [kdd.ics.uci.edu](https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html)

Using a KNeighborsClassifier(n_neighbors=k, metric=metric,
weights=’distance’), with different values for the number of neigh-
bors k and at least 3 different metrics. (see Lab3 for example).

Compare the following text vectorization strategies and discuss the
results when using:
* raw word frequencies
* Term Frequency-Inverse Document Frequency (TF-IDF)
* encoding/embedding method of your choice (e.g., BOW, Word2Vec,
GloVe)

Use the above obtained high-dimensional data set (with one of the
representations - raw word frequencies, TF-IDF, etc.) and apply di-
mensionality reduction techniques (e.g., PCA, t-SNE, UMAP). Again,
analyze the performance of a classification model of your choice for the
initial and for the reduced data produced using two different dimen-
sionality reduction techniques.

# 1. Data set loading and summary
 In this section, we load the dataset and also print a small summary of the dataset
 We also import all the things we might need


In [ ]:
import nltk
# nltk.data.path.append('C:\\Users\\Magda\\AppData\\Roaming\\nltk_data')
# The NLTK library alread containes the Reuters-21578 data set
#  and all we have to do is import it from NLTK
from nltk.corpus import reuters
# import nltk.data

# We also import a dataset of 'stop words' which are general words
#  such as 'the', 'of', 'and' and are usually meaningless.
# We use this stopwords list to remove them from the Reuters data set.
from nltk.corpus import stopwords

# To use the Reuters-21578 from within NLTK, we have to download it first.
nltk.download("reuters") # run only once, after that it will be ignored anyway

# To use the stopwords from within NLTK, we have to download them first.
nltk.download("stopwords") # run only once, after that it will be ignored anyway

# WordNet corpus required by the WordNetLemmatizer
nltk.download('wordnet') # run only once, after that it will be ignored anyway

# Data for NLTK word tokenizer, otherwise word_tokenize() will not work
nltk.download('punkt') # run only once, after that it will be ignored anyway
nltk.download('punkt_tab') # run only once, after that it will be ignored anyway

[nltk_data] Downloading package reuters to /home/acasa/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to /home/acasa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/acasa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/acasa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/acasa/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [29]:
# Print a quick summary of the Reuters-21578 dataset
fileIds = reuters.fileids()
print("Reuters-21578 number of tags:", len(fileIds))
print("Reuters-21578 number of documents:", len(fileIds))

Reuters-21578 number of tags: 10788
Reuters-21578 number of documents: 10788


In [30]:
# Find all the train documents and print their number
trainDocuments = list(filter(lambda doc: doc.startswith("train"), fileIds))
print("Reuters-21578 number of train documents:", len(trainDocuments))

# Find all the test documents and print their number
testDocuments = list(filter(lambda doc: doc.startswith("test"), fileIds))
print("Reuters-21578 number of test documents:", len(testDocuments))

# Print the number of categories from the Reuters-21578 dataset
categories = reuters.categories()
print("Reuters-21578 number of categories:", len(categories))

Reuters-21578 number of train documents: 7769
Reuters-21578 number of test documents: 3019
Reuters-21578 number of categories: 90


In [31]:
import cmd # for printing a list as a column rather than as a row

# Print the categories from the Reuters-21578 data set
# We use the 'cmd' package to print the list based on a fixed width, otherwise the list
# of categories would print on one single very long row, which is hard to read
cli = cmd.Cmd()
print("Reuters-21578 categories:")
print(cli.columnize(categories, displaywidth = 80))

Reuters-21578 categories:
acq          cpi            hog          meal-feed     platinum  soy-oil        
alum         cpu            housing      money-fx      potato    soybean        
barley       crude          income       money-supply  propane   strategic-metal
bop          dfl            instal-debt  naphtha       rand      sugar          
carcass      dlr            interest     nat-gas       rape-oil  sun-meal       
castor-oil   dmk            ipi          nickel        rapeseed  sun-oil        
cocoa        earn           iron-steel   nkr           reserves  sunseed        
coconut      fuel           jet          nzdlr         retail    tea            
coconut-oil  gas            jobs         oat           rice      tin            
coffee       gnp            l-cattle     oilseed       rubber    trade          
copper       gold           lead         orange        rye       veg-oil        
copra-cake   grain          lei          palladium     ship      wheat          
co

# 2. Document tokenizing
 In this section, we define a tokenize() helper function to lowercase and tokenize the text, then stem each token.

In [32]:
# Get the stopwords corresponding to the English language
stopwordsList = stopwords.words("english")
stopWordsEN = set(stopwordsList)

# Print the stop words in multiple columns, where the total width of all the columns is 80
print("Stop words:")
print(cli.columnize(stopwordsList, displaywidth = 80))

Stop words:
a        but       hadn't   in        needn      she         they'll  when      
about    by        has      into      needn't    she'd       they're  where     
above    can       hasn     is        no         she'll      they've  which     
after    couldn    hasn't   isn       nor        she's       this     while     
again    couldn't  have     isn't     not        should      those    who       
against  d         haven    it        now        shouldn     through  whom      
ain      did       haven't  it'd      o          shouldn't   to       why       
all      didn      having   it'll     of         should've   too      will      
am       didn't    he       it's      off        so          under    with      
an       do        he'd     its       on         some        until    won       
and      does      he'll    itself    once       such        up       won't     
any      doesn     her      i've      only       t           ve       wouldn    
are      doesn't

In [33]:
# from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Helper function that tokenizes, lemmatizes and removes stopwords from a single document.
#    Then the function returns also a document, but in processed and tokenized format.
#    The main tokenization is made with the NLTK 'word_tokenize()' function.
# Instead of stemming, we lemmatize, which is similar to stemming but offers better results
# at the expense of higher CPU demand.
#    The purpose of this function is to generate a cleaner, tokenized document for the TF-IDF
# algorithm, which, in the end, requires a list of documents.
#    This function can produce a single document and will be used a loop.
def tokenizeDocument(document):
    # First, tokenize the document passed as argument. We use NLTK 'word_tokenize()' which
    # return a list of strings
    tokens = nltk.word_tokenize(document, language = 'english')

    # Filter tokens:
    # - only alphabetical words are allowed
    # - only words wich are not stopwords are allowed
    lemmatizer = WordNetLemmatizer()
    processedTokens = []
    processedToken = ""
    for token in tokens:
        if (token.isalpha() and token not in stopWordsEN):
            processedToken = lemmatizer.lemmatize(token.lower())
            processedTokens.append(processedToken)

    # Join the processed tokens into a single string, this string will represent
    # a single document.
    #    If we just join the tokens directly, they will just create a single very large word,
    # but we are joining them in such a way that they have a space between them, by using
    # " ".join(list) - e.g. join the strings from a list and put a space between them.
    #    In this way, the single string obtained will be recognized by the TF-IDF algorithm
    # as an document of words, not a single large word (spaces are the standard delimiters that
    # the TfidfVectorizer uses).
    finalDocument = " ".join(processedTokens) # note the space in the string (it's not an empty string)

    return finalDocument

# 3. Vectorize the Reuters-21578 data set by using TF-IDF

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = []
for fileId in fileIds:
    documents.append(reuters.raw(fileId))

processedDocuments = []
for doc in documents:
    processedDocuments.append(tokenizeDocument(doc))

vectorizer = TfidfVectorizer(max_df = 0.85, min_df = 2)

# Fit and then transform
tfIdfMatrix = vectorizer.fit_transform(processedDocuments)

print(tfIdfMatrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 531388 stored elements and shape (10788, 14157)>
  Coords	Values
  (0, 840)	0.08383479033397855
  (0, 4564)	0.09810947267937536
  (0, 4713)	0.07358504462157547
  (0, 3129)	0.07096776365365834
  (0, 5144)	0.034106087336237795
  (0, 10838)	0.05852669195279093
  (0, 8232)	0.04702388083660899
  (0, 12909)	0.32190522480092104
  (0, 5126)	0.04549954376580683
  (0, 579)	0.10632447902300628
  (0, 6708)	0.2996368734952329
  (0, 10106)	0.030222980594274796
  (0, 538)	0.0619506956598169
  (0, 7643)	0.030289969713393424
  (0, 839)	0.04153421639970694
  (0, 4565)	0.04050281377317544
  (0, 8355)	0.028614909948297303
  (0, 10970)	0.04060974363583497
  (0, 2895)	0.022506364378516868
  (0, 3991)	0.05166717162857884
  (0, 1748)	0.17225580121737882
  (0, 8734)	0.08674770081875548
  (0, 11062)	0.1540175036197175
  (0, 12708)	0.05461908075671201
  (0, 12827)	0.021140060382043697
  :	:
  (10786, 8791)	0.16546822694547178
  (10786, 8446)	0.1375048

We decide to do fitting and transforming over the processed document:
    <blockquote>Fitting - will tell the TfidfVectorizer to learn the vocabulary and the importance of the words </blockquote>
    <blockquote>Transform - will convert the document into numerical vectors with the information it learned </blockquote>

Both of these are important so we can get a TfIDF matrix that we can use more later.

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the date into training and testing sets
# We're first creating labels (category) for each document 
labels = [reuters.categories(fileId)[0] for fileId in fileIds]

# Doing the actual splitting
X_train, X_test, y_train, y_test = train_test_split(tfIdfMatrix, labels, test_size=0.2, random_state=42)
# The training set is going to be used for teaching the model patterns in the data
# And the test set is used to ensure the model performs well on unseen data (so to ensure its
# generalized well)
# In our case we decided to keep just 20% of the data for testing 
# and i sued random state to be able to have the same split every time 

print(f"Training data size: {X_train.shape[0]}")
print(f"Test data size: {X_test.shape[0]}")

Training data size: 8630
Test data size: 2158


# 3.1 Comparing with different mentrics and k-values

In [36]:
from sklearn.neighbors import KNeighborsClassifier
import time

# Training KNN with different metrics using for in for
# We will be using the following:
# euclidian distance - which is the default for this 
# cosine distance - which can be informative when working with text
# manhattan distance - an alternative to euclidean ?

metrics =['euclidean', 'cosine', 'manhattan']
k_values = [3, 5, 7]

results = []

for metric in metrics:
    for k in k_values:
        start_time = time.time()

        # We're initializing the KNN Classifier with the current metric and k value
        knn = KNeighborsClassifier(n_neighbors=k,metric=metric,weights='distance')

        # We're fitting the model 
        knn.fit(X_train, y_train)

        # We're evaluating the model and tracking the results
        accuracy = knn.score(X_test,y_test)
        results.append((metric, k, accuracy))

        end_time = time.time()

        print(f"Metric: {metric}, k: {k}, Accuracy: {accuracy:.4f}, Time: {end_time-start_time:.4f} seconds")

Metric: euclidean, k: 3, Accuracy: 0.6409, Time: 0.7418 seconds
Metric: euclidean, k: 5, Accuracy: 0.5950, Time: 0.7020 seconds
Metric: euclidean, k: 7, Accuracy: 0.5635, Time: 0.6842 seconds
Metric: cosine, k: 3, Accuracy: 0.8230, Time: 0.6795 seconds
Metric: cosine, k: 5, Accuracy: 0.8369, Time: 0.6812 seconds
Metric: cosine, k: 7, Accuracy: 0.8448, Time: 0.6873 seconds
Metric: manhattan, k: 3, Accuracy: 0.4272, Time: 1.2846 seconds
Metric: manhattan, k: 5, Accuracy: 0.4106, Time: 1.1764 seconds
Metric: manhattan, k: 7, Accuracy: 0.4082, Time: 1.0433 seconds


We will go into an explanation about how the choice of distance metric and the number of neighbors affects the performance of KNN in terms of accuracy and time.

<h3>1. Accuracy comparison</h3>
We can notice how <b>Cosine metric</b> gives us the best accuracy across all of the k values compared to the other two choices.
The other two can be considered significantly lower, with <b>Euclidean</b> performing a little bit better than <b>Manhattan</b> still.

<h3>1.1 The reason why Cosine is better</h3>
Cosine is used for text and high-dimension data, because, as we know, it measures the similarity between two documents by calculating the cosine of the angle between the vectors.
As cosine performs the best for us, we can assume we have high-dimensional features and cosine better distincts between classes compared to the other ones.

<h3>1.2 The reason Manhattan performs poorly</h3>
Manhattan distance measures the sum of absolute differences of coordinates. It is usually used in grid-like structures, like city maps or certain robot motion planning scenarios. So in high-dimensional places it tends to fail to capture the actual closeness of points (and it leads to poor classification)

<h3>2. Effect of k on Accuracy</h3>
We notice that in Euclidean and Manhattan increasing the value of k will reduce accuracy, unlike cosine that increases with a higher k value, peaking at almost 85% with k=7.

<h3>3. Time comparison</h3>
Cosine and Euclidean are faster. Manhattan is slower (taking nearly double the time) because calculating absolute differences takes a lot longer compared to squared differences (Euclidean) or dot products (Cosine)


# 3.2 Comparring with raw frequency (BOW?)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

# We're using count vectorizer to get the raw frequency for the words
count_vectorizer = CountVectorizer(max_df=0.85, min_df=2)

# Fitting and transforming raw word frequency
frequency_matrix = count_vectorizer.fit_transform(processedDocuments)

# Splitting the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(frequency_matrix, labels, test_size=0.2, random_state=42)

for metric in metrics:
    for k in k_values:
        start_time = time.time()
        knn = KNeighborsClassifier(n_neighbors=k, metric=metric, weights='distance')
        knn.fit(X_train,y_train)
        accuracy = knn.score(X_test,y_test)
        end_time = time.time()
        print(f"Raw Word Frequency - Metric: {metric}, k: {k}, Accuracy: {accuracy:.4f}, Time: {end_time-start_time:.4f} seconds")

Raw Word Frequency - Metric: euclidean, k: 3, Accuracy: 0.7539, Time: 0.8411 seconds
Raw Word Frequency - Metric: euclidean, k: 5, Accuracy: 0.7349, Time: 0.8233 seconds
Raw Word Frequency - Metric: euclidean, k: 7, Accuracy: 0.7210, Time: 0.8062 seconds
Raw Word Frequency - Metric: cosine, k: 3, Accuracy: 0.8471, Time: 0.8109 seconds
Raw Word Frequency - Metric: cosine, k: 5, Accuracy: 0.8452, Time: 0.8099 seconds
Raw Word Frequency - Metric: cosine, k: 7, Accuracy: 0.8494, Time: 0.8087 seconds
Raw Word Frequency - Metric: manhattan, k: 3, Accuracy: 0.6186, Time: 2.8101 seconds
Raw Word Frequency - Metric: manhattan, k: 5, Accuracy: 0.5931, Time: 2.8375 seconds
Raw Word Frequency - Metric: manhattan, k: 7, Accuracy: 0.5737, Time: 3.1083 seconds


We will go into the explanations for using raw word frequency extracted with CountVectorizer

<h3>1. Accuracy comparison</h3>
We can once again notice how Cosine outperforms the other two options we had (Euclidean and Manhattan). Now with raw word frequency we can say that Euclidean performs more decently, but Manhattan is still falling behind 

<h3>2. Effect of k on Accuracy</h3>
It performs similar to above. we can see Euclidean and Manhattan decreasing in accuracy with the increase of k value, whereas Cosine remains quite stable.

<h3>3. Time comparison</h3>
It's noticeable, just like before, how Euclidean and Cosine run a lot faster, again compared to Manhattan that is very slow.

Again, we arrive at the conclusion that Cosine is our best option.

# 4. Apply the PCA and t-SNE algorithms for reducing the dimensionality of the data set


In [ ]:
# This code will take about 1min 30sec to finish.

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import numpy as np

# Convert sparse matrix to dense matrix
denseMatrix = tfIdfMatrix.toarray()

# Reduce the dimensionality of the data set by using the PCA algorithm. We pass as main argument
# the number of principal components: this is bassicaly the new dimension of the
# converted data. The lower the dimension, the simpler the data becomes, but we also risk
# loosing important information from the data.
pca = PCA(n_components = 2)
reducedPCAMatrix = pca.fit_transform(denseMatrix)
print("Shape of PCA-reduced matrix:", reducedPCAMatrix.shape)
print("Explained variance ratio of PCA:", np.sum(pca.explained_variance_ratio_))

# Reduce the dimensionality of the data set by using the t-SNE algorithm.
#    We use a very small number of principal components because the t-SNE algorithm uses by default
# the 'barnes_hut' algorithm for calculating the gradient and that requires a number of principal
# components smaller than 4.
#    If we use the 'exact' algorithm for gradient calculation, the performance is going to be poor. 
tsne = TSNE(n_components = 2, random_state = 0)
reducedTSNEMatrix = tsne.fit_transform(denseMatrix)
print("Shape of TSNE-reduced matrix:", reducedTSNEMatrix.shape)

Shape of PCA-reduced matrix: (10788, 2)
Explained variance ratio of PCA: 0.07506011215752095
Shape of TSNE-reduced matrix: (10788, 2)


In [39]:
# Print the first few PCA-reduced documents and t-SNE-reduced documents
for docIndex in range(20):
    # Print the current PCA-reduced document
    print("\nPCA-reduced Vector for Example Document:")
    print(reducedPCAMatrix[docIndex])

    # Print the current t-SNE-reduced document
    print("\nt-SNE-reduced Vector for Example Document:")
    print(reducedTSNEMatrix[docIndex])


PCA-reduced Vector for Example Document:
[-0.17973043  0.0836326 ]

t-SNE-reduced Vector for Example Document:
[-64.87492   35.620476]

PCA-reduced Vector for Example Document:
[-0.13454316  0.03090986]

t-SNE-reduced Vector for Example Document:
[ 56.724167 -41.84875 ]

PCA-reduced Vector for Example Document:
[-0.14066232  0.03542713]

t-SNE-reduced Vector for Example Document:
[ 27.507524 -62.834003]

PCA-reduced Vector for Example Document:
[-0.178002    0.08741341]

t-SNE-reduced Vector for Example Document:
[  4.270859 -28.907513]

PCA-reduced Vector for Example Document:
[-0.1588988   0.04589015]

t-SNE-reduced Vector for Example Document:
[-30.512508 -15.358434]

PCA-reduced Vector for Example Document:
[-0.15298924  0.00391352]

t-SNE-reduced Vector for Example Document:
[-23.92899   15.120278]

PCA-reduced Vector for Example Document:
[-0.16472713  0.02986478]

t-SNE-reduced Vector for Example Document:
[65.772194 32.331203]

PCA-reduced Vector for Example Document:
[-0.1490